In [14]:
import numpy as np
import math
import sys

class Sample:
    "Represenation of data samples"
    def __init__(self, xclass, values, identity, attributes=None):
        self.xclass = xclass
        if attributes == None:
            self.attribute = values
        else:
            self.attribute = dict(zip(attributes, values))
        self.identity = identity
        
    def getClass(self):
        return self.xclass
    
    def getAttributes(self):
        return self.attribute
    
    def getNbrAttributes(self):
        return len(self.attribute)
    
    def getAttributeValue(self,attribute):
        return self.attribute[attribute]
    
    def getIdentity(self):
        return self.identity

CancerAttributes = (
                'Clump Thickness',
                'Uniformity of Cell Size',
                'Uniformity of Cell Shape',
                'Marginal Adhesion',
                'Single Epithelial Cell Size',
                'Bare Nuclei',
                'Bland Chromatin',
                'Normal Nucleoli',
                'Mitoses'
              )

In [15]:
## Hyperparameters
sys.setrecursionlimit(10000)
soF = 1 # number of features randomly chosen
nbrTrees = 100 # number of trees in the forest
MAXDEPTH = 10#math.inf

In [16]:
## Psuedo Code
'''
Precondition: A training set S := (x1, y1), . . . ,(xn, yn), features F, and number
of trees in forest B.
1 function RandomForest(S , F)
    2 H ← ∅
    3 for i ∈ 1, . . . , B do
        4 S(i) ← A bootstrap sample from S
        5 hi ← RandomizedTreeLearn(S(i), F)
        6 H ← H ∪ {hi}
    7 end for
    8 return H
9 end function

10 function RandomizedTreeLearn(S , F)
    11 At each node:
        12 f ← very small subset of F
        13 Split on best feature in f
    14 return The learned tree
15 end function
'''

def RandomForest(S, F=None, SF=True):
    if F == None:
        F = range(S[0].getNbrAttributes())
    B = nbrTrees                    # number of trees in forest
    H = []
    for i in range(B):
        S_i = BootstrapSample(S)
        h_i = RandomizedTreeLearn(S_i, F, SF)
        H.append(h_i)
    return H

def RandomizedTreeLearn(S, F, singleFeature, maxdepth = MAXDEPTH): # here S is a subsample 
    def buildBranch(dataset, default, attributes):
        if not dataset:
            return TreeLeaf(default)
        allClass,theClass = allAnyClass(dataset)
        if allClass:
            return TreeLeaf(theClass)
        return RandomizedTreeLearn(dataset, attributes, singleFeature, maxdepth-1)

    default = mostCommon(S)
    if maxdepth < 1:
        return TreeLeaf(default)
    f = FeatureSubsample(F, singleFeature) # feature subsample
    a,v = bestFeature(S, f)
    attributesLeftL = F
    attributesLeftR = F
    dataL = select(S, a, v, False)
    dataR = select(S, a, v, True)
    branches = [buildBranch(dataL, default, attributesLeftL),
                buildBranch(dataR, default, attributesLeftR)]
    return TreeNode(a, branches, default, v)

def BootstrapSample(S):
    soS = 10                   # size of subsample
    return np.random.choice(S, soS).tolist()

def FeatureSubsample(F, singleFeature):
    soS = soF
    if not singleFeature:
        soS = int(math.log(len(F)+1,2))
    return np.random.choice(F, soS).tolist()

class TreeNode:
    def __init__(self, attribute, branches, default, value):
        self.attribute = attribute
        self.branches = branches
        self.default = default
        self.value = value
        
    def __repr__(self):
        accum = str(self.attribute) + '('
        for branch in self.branches:
            accum += str(branch)
        return accum + ')'
    
class TreeLeaf:
    def __init__(self, cvalue):
        self.cvalue = cvalue

    def __repr__(self):
        return str(self.cvalue)

def getAttributeValues(S,attribute):
    values = []
    for x in S:
        val = x.getAttributeValue(attribute)
        if not val in values:
            values += [val]
    return sorted(values)
    
def bestFeature(S, features):
    if len(S) <= 1:
        return None
    gains = [(averageGain(S, f, v), f, v) for f in features for v in getAttributeValues(S,f)]
    return max(gains, key=lambda x: x[0])[1:]

def mostCommon(S):
    classes = list(set([x.getClass() for x in S]))
    return classes[np.argmax([len([x for x in S if x.getClass()==c]) for c in classes])]

def averageGain(dataset, attribute, value):
    weighted = 0.0
    subsetOver = select(dataset, attribute, value, True)
    weighted += entropy(subsetOver) * len(subsetOver)
    subsetUnder = select(dataset, attribute, value, False)
    weighted += entropy(subsetUnder) * len(subsetUnder)
    return entropy(dataset) - weighted/len(dataset)

def entropy(dataset):
    "Calculate the entropy of a dataset"
    classes = list(set([x.getClass() for x in dataset]))
    n = len(dataset)
    entropy = 0
    for c in classes:
        nclass = len([x for x in dataset if x.getClass() == c])
        if nclass == 0:
            continue
        else:
            entropy -= float(nclass)/n * math.log(float(nclass)/n,2)
    return entropy
    
def select(dataset, attribute, value, takeOver):
    #print('value', value)
    #print('attribute', attribute)
    #print(dataset[0].attribute[attribute])
    return [x for x in dataset if (x.getAttributeValue(attribute) >= value) == takeOver]

def allFromClass(dataset,c):
    "Check if all samples are from class c"
    return all([x.getClass() == c for x in dataset])

def allAnyClass(dataset):
    if len(dataset) == 0:
        return (False,0)
    "Check if all samples are from the same class"
    c = dataset[0].getClass()
    if allFromClass(dataset,c):
        return (True,c)
    return (False,0)

In [17]:
def classify(tree, sample):
    "Classify a sample using the given decition tree"
    if isinstance(tree, TreeLeaf):
        return tree.cvalue
    return classify(tree.branches[int(sample.getAttributeValue(tree.attribute)>=tree.value)], sample)

def classifyForest(forest, sample):
    "Classify a sample using the given decition tree"
    classifications = []
    for tree in forest:
        classifications += [classify(tree, sample)]
    counts = [(c,classifications.count(c)) for c in set(classifications)]
    return sorted(counts,key=lambda x: x[1])[-1][0]
    

def check(tree, testdata):
    "Measure fraction of correctly classified samples"
    correct = 0
    for x in testdata:
        if classify(tree, x) == x.getClass():
            correct += 1
    return float(correct)/len(testdata)

def checkForest(forest, testdata):
    "Measure fraction of correctly classified samples"
    correct = 0
    for x in testdata:
        if classifyForest(forest, x) == x.getClass():
            correct += 1
    return float(correct)/len(testdata)

In [18]:
ITERATE = 10

def checkOnData(data):
    S = chooseData(data)
    N = len(S)
    cumulErrorSelection = 0
    cumulErrorF1 = 0
    cumulErrorF2 = 0
    cumulSpecificError = 0
    for i in range(ITERATE):
        np.random.shuffle(S)
        train, test = S[:int(0.9*N)],S[int(0.9*N):]
        H = RandomForest(train)
        H2 = RandomForest(train,SF=False)
        cumulSpecificError += sum([1-check(h,test) for h in H])/nbrTrees
        errorF1 = 1-checkForest(H,test)
        errorF2 = 1-checkForest(H2,test)
        cumulErrorSelection += min(errorF1,errorF2)
        cumulErrorF1 += errorF1
        
    finalErrorF1 = cumulErrorF1/ITERATE
    finalErrorSelection = cumulErrorSelection/ITERATE
    finalSpecificError = cumulSpecificError/ITERATE
    
    print("Data :", data)
    print("Number of input :", str(S[0].getNbrAttributes()))
    print("Number of data point :", str(len(S)))
    print("Error rate with selection :", str(finalErrorSelection))
    print("Error rate with single input :", str(finalErrorF1))
    print("Error rate with individual trees :", str(finalSpecificError))
    print()
    

In [19]:
import numpy as np
import pandas as pd

def chooseData(data):
    prefix = 'datasets/'
    if data == "glass":
        data =np.genfromtxt(prefix + 'glass.data',dtype=None,delimiter=",")
        return [Sample(x[-1],[x[i] for i in range(1,len(x)-1)],x[0]) for x in data]
    
    elif data == "ecoli":
        data = np.genfromtxt(prefix + 'ecoli.data',dtype=None)
        return [Sample(x[-1],[x[i] for i in range(1,len(x)-1)],x[0]) for x in data]
    
    elif data == "diabetes":
        data = np.genfromtxt(prefix + 'pima-indians-diabetes.data',dtype=None,delimiter=",")
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "sonar":
        data = np.genfromtxt(prefix + 'sonar.all-data',dtype=None,delimiter=",")
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "vowel":
        data = np.genfromtxt(prefix + 'vowel-context.data',dtype=None)
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "ionosphere":
        data = np.genfromtxt(prefix + 'ionosphere.data',dtype=None, delimiter=',')
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "vehicle":
        data = np.genfromtxt(prefix + 'vehicle.data',dtype=None, delimiter=',')
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "german":
        data = np.genfromtxt(prefix + 'german.data-numeric',dtype=None)
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "image":
        data = np.genfromtxt(prefix + 'segment.dat',dtype=None)
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "cancer":
        data = np.genfromtxt(prefix + 'breast-cancer-wisconsin.data',delimiter=",",dtype=int)
        return [Sample(x[-1],[x[i] for i in range(1,len(x)-1)],x[0]) for x in data]
    
    elif data == "votes":
        data = np.genfromtxt(prefix + "house-votes-84.data",dtype=str,delimiter=',')
        #data = [list(x) for x in data if not any([y == '?' for y in x])]
        data = [[int(y == 'y' or y == 'democrat') - int(y == '?') for y in x] for x in data] # to numerical values
        dataIndex = np.where(np.matrix(data)[:,-1] > -1)[0]
        data = [data[idx] for idx in list(dataIndex)]
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]
    
    elif data == "liver":
        data = np.genfromtxt(prefix + "bupa.data",dtype=None,delimiter=',')
        return [Sample(data[x][-1],[data[x][i] for i in range(len(data[x])-1)],x) for x in range(len(data))]

In [20]:
checkOnData("glass")
checkOnData("cancer")
checkOnData("diabetes")
checkOnData("sonar")
checkOnData("vowel")
checkOnData("ionosphere")
checkOnData("vehicle")
checkOnData("german")
checkOnData("image")
checkOnData("ecoli")
checkOnData("votes")
checkOnData("liver")

('Data :', 'glass')
('Number of input :', '9')
('Number of data point :', '214')
('Error rate with selection :', '0.436363636364')
('Error rate with single input :', '0.463636363636')
('Error rate with individual trees :', '0.640772727273')
()
('Data :', 'cancer')
('Number of input :', '9')
('Number of data point :', '699')
('Error rate with selection :', '0.0671428571429')
('Error rate with single input :', '0.0728571428571')
('Error rate with individual trees :', '0.150442857143')
()
('Data :', 'diabetes')
('Number of input :', '8')
('Number of data point :', '768')
('Error rate with selection :', '0.309090909091')
('Error rate with single input :', '0.322077922078')
('Error rate with individual trees :', '0.389818181818')
()
('Data :', 'sonar')
('Number of input :', '60')
('Number of data point :', '208')
('Error rate with selection :', '0.209523809524')
('Error rate with single input :', '0.242857142857')
('Error rate with individual trees :', '0.455761904762')
()
('Data :', 'vowel